In [3]:
ls /home/jovyan/

uncanny_notebook.ipynb  Untitled.ipynb  work



### Todo
* check with phd-data dir and conf paper for up to date ness

# License Evaluation Contract Representation

In the following notebook the examples of my thesis are accumulated in such a state as to make 

### Natural legal language example (abbreviated)

Example:

This license is an example evaluation license.

LICENSEE - The University.  
SUBLICENSEE - A student as set force in the appendix. (array of persons)  
ARBITER - An arbiter or oracle that decides incase of disputes.  Can be a natural or legal person or a machine. 
Art.  2, 3 and 4 especially areevaluated by the arbiter in case of disputes.  
ASSET X - An asset to be licensed.  

* Article 1. The  Licensor  grants  the  Licensee  a  license  to use  and  evaluate  asset  X  and  grant  sublicensesamong  group  Y,  for  use  and  evaluation.  
This grant is in exchange for a licensing fee.
* Article 2. (optional) The (Sub)Licensee is commissioned to publish  comments  about  the  use  of  the  product.  
This allows publication of comments but also re-quires them.
* Article 3. The (Sub)Licensee must not publish comments ofthe use and evaluation of the Product without theapproval  of  the  Licensor;  
the  approval must  beobtained before the publication. If the Licensee publishes results of the evaluation of the Product without approval from the Licensor, the Licensee has 24 h to remove the material.
* Article 4. This license terminates automatically if  the  (Sub)Licensee  breaches  this  Agreement. Breach obliges the licensee to pay a fee to Licensor for Breach of the Licensing Terms.

## Soldity

Soliditiy can be compiled, but for testing its functions, it would have to be copied into remix ide. Also it should be noted that for abbrevation purposes, this contract is not necessarily secure or complete from a usability point of view, such as convenience functions and similar might be missing. This is due to the fact that it would otherwise be too long for publication.

pragma solidity ^0.6.7;
pragma experimental ABIEncoderV2;
import "github.com/OpenZeppelin/openzeppelin-contracts/contracts/utils/EnumerableMap.sol";

contract LicenseContract {
    //using EnumerableMap for EnumerableMap.UintToAddressMap;
    bytes32 work_hash; string name;
    address payable licensor;
    address licensee;	
    //  EnumerableMap.UintToAddressMap sublicensees;
    address payable arbiter;
    bool breachFeePaid;
    
event Status(LicenseProps);
    
struct LicenseProps {
    int id;
    int licenseFee;
    uint breachFee;
    bool isCommissioned;
    bool publicationIsApproved;
    bool requiresComments;
    uint timeToRemove;
    bool triggeredTimeToRemove;
    bool licenseBreached;}

mapping (uint => LicenseProps) licenses;
uint[] public licensesList;
uint numLicenses;
//EnumerableMap.UintToAddressMap memory _sublicensees,
modifier onlyBy(address _account)
    {require(msg.sender == _account,
          "Sender not authorized.");
        _;}

function newLicense(int _id, int _licenseFee, uint _breachFee, bool _isCommissioned, bool _publicationIsApproved, bool _requiresComments, uint _timeToRemove, bool _triggeredTimeToRemove, bool _licenseBreached) public returns (uint licenseID) {
        licenses[licenseID] = LicenseProps(_id, _licenseFee, _breachFee, _isCommissioned, _publicationIsApproved, _requiresComments, _timeToRemove, _triggeredTimeToRemove, _licenseBreached);
        licenseID = numLicenses++;
        // sublicensees = _sublicensees;
        return licenseID; }

function commissionComments(uint _licenseID) public {
    LicenseProps storage l = licenses[_licenseID];
    l.publicationIsApproved = true;
    l.requiresComments = true;
	l.isCommissioned = true; }

function grantApproval(uint _licenseID) public {
    LicenseProps storage l = licenses[_licenseID];
    l.publicationIsApproved = true; }

function evalPublication(uint _licenseID, bool isPublished) public {
    LicenseProps storage l = licenses[_licenseID];
    if (isPublished) {
        if ((isPublished && !l.isCommissioned || !l.publicationIsApproved) && !l.triggeredTimeToRemove) {
            l.timeToRemove = now;
            l.triggeredTimeToRemove = true;
        } else if (l.triggeredTimeToRemove && now > l.timeToRemove + 1 days ) {
            declareBreach(_licenseID);
        }    }
    emit Status(l); }

function declareRemoved(uint _licenseID) public onlyBy(arbiter) {
    LicenseProps storage l = licenses[_licenseID];
    l.timeToRemove = 0;
    l.triggeredTimeToRemove = false;
}

function declareBreach(uint _licenseID) public onlyBy(arbiter) {
    LicenseProps storage l = licenses[_licenseID];
	l.licenseBreached = true;
	if (!breachFeePaid) {
	    breachFeePaid = false;
	    licensor.transfer(l.breachFee);
	    }     } }


In [32]:
find / -iname solc

find: ‘/etc/ssl/private’: Permission denied
find: ‘/proc/tty/driver’: Permission denied
find: ‘/var/cache/apt/archives/partial’: Permission denied
find: ‘/var/cache/ldconfig’: Permission denied
find: ‘/root’: Permission denied



## ACE / Attempto

In [5]:
export ACETEXTPROLOG="An arbiter is a person.\
A n:licensor is a person.\
A licensee is a person.\
A n:sublicensee is a person. \
A n:licensor commissions a licensee. \
A n:duration-to-remove-comments is 24 hours. \
A n:license-fee is an amount. \
If there is a n:licensor and there is a licensee then a n:license-fee is required. \
If the n:license-fee is paid then the licensee receives a license. \
If there is a license then the licensee can v:sublicense. \
If there is a license then the licensee can comment. \
If the licensee may comment then the n:licensor may approve. \
If a n:licensor commissions the licensee then a licensee has to comment. \
If the n:licensor does not approve then the licensee may not comment. \
If the licensee may not publish then the comments have to be removed within 24 hours. \
If the comments have to be removed and are not removed within 24 hours then the license is breached. \
If the license is breached then a n:breach-fee has to be paid."

In [6]:
echo $ACETEXTPROLOG

An arbiter is a person.A n:licensor is a person.A licensee is a person.A n:sublicensee is a person. A n:licensor commissions a licensee. A n:duration-to-remove-comments is 24 hours. A n:license-fee is an amount. If there is a n:licensor and there is a licensee then a n:license-fee is required. If the n:license-fee is paid then the licensee receives a license. If there is a license then the licensee can v:sublicense. If there is a license then the licensee can comment. If the licensee may comment then the n:licensor may approve. If a n:licensor commissions the licensee then a licensee has to comment. If the n:licensor does not approve then the licensee may not comment. If the licensee may not publish then the comments have to be removed within 24 hours. If the comments have to be removed and are not removed within 24 hours then the license is breached. If the license is breached then a n:breach-fee has to be paid.



In [13]:
acetext=$(<ace.txt)

In [16]:
cat "$acetext"

cat: 'An arbiter is a person.'$'\n''A n:licensor is a person.'$'\n''A licensee is a person.'$'\n''A n:sublicensee is a person.'$'\n''A n:licensor commissions a licensee.'$'\n''A n:duration-to-remove-comments is 24 hours.'$'\n''A n:license-fee is an amount.'$'\n''/* next part for similarity to prolog */'$'\n''/* If to be an offer by the n:licensor. */'$'\n''If there is a n:licensor and there is a licensee then a n:license-fee is required.'$'\n''If the n:license-fee is paid then the licensee receives a license.'$'\n''If there is a license then the licensee can v:sublicense'$'\n''/* gives licenses omitted */'$'\n''If there is a license then the licensee can comment.'$'\n''If the licensee may comment then the n:licensor may approve.'$'\n''If a n:licensor commissions the licensee then a licensee has to comment.'$'\n''If the n:licensor does not approve then the licensee may not comment.'$'\n''If the licensee may not publish then the comments have to be removed within 24 hours.'$'\n''If the c

Below is an exexcution of the ACE example, with the Ace Parsing Enginge (APE). It is currently read from a file and not from a notebook cell, but the file can be modified and also output of ape changed to other formats. < Link to APE options >

In [24]:
/home/jovyan/bin/ape.bin -file ace.txt -solo tokens

[[^,'An',arbiter,is,a,person,'.'],[^,'A',n,:,licensor,is,a,person,'.'],[^,'A',licensee,is,a,person,'.'],[^,'A',n,:,sublicensee,is,a,person,'.'],[^,'A',n,:,licensor,commissions,a,licensee,'.'],[^,'A',n,:,'duration-to-remove-comments',is,24,hours,'.'],[^,'A',n,:,'license-fee',is,an,amount,'.'],[^,'If',there,is,a,n,:,licensor,and,there,is,a,licensee,then,a,n,:,'license-fee',is,required,'.'],[^,'If',the,n,:,'license-fee',is,paid,then,the,licensee,receives,a,license,'.'],[^,'If',there,is,a,license,then,the,licensee,can,v,:,sublicense,'.'],[^,'If',there,is,a,license,then,the,licensee,can,comment,'.'],[^,'If',the,licensee,may,comment,then,the,n,:,licensor,may,approve,'.'],[^,'If',a,n,:,licensor,commissions,the,licensee,then,a,licensee,has,to,comment,'.'],[^,'If',the,n,:,licensor,does,not,approve,then,the,licensee,may,not,comment,'.'],[^,'If',the,licensee,may,not,publish,then,the,comments,have,to,be,removed,within,24,hours,'.'],[^,'If',the,comments,have,to,be,removed,and,are,not,removed,within

In [26]:
/home/jovyan/bin/ape.bin -file ace.txt -solo paraphrase

There is an arbiter X1.
The arbiter X1 is a person.

There is a n:licensor X1.
The n:licensor X1 is a person.

There is a licensee X1.
The licensee X1 is a person.

There is a n:sublicensee X1.
The n:sublicensee X1 is a person.

There is a n:licensor X1.
The n:licensor X1 commissions a licensee X2.
There is a n:license-fee X3.
The n:license-fee X3 is an amount.
If the n:license-fee X3 is paid then the licensee X2 receives a license.
If there is a license then it is possible that the licensee X2 v:sublicense .
If there is a license then it is possible that the licensee X2 comments.
If it is admissible that the licensee X2 comments then it is admissible that the n:licensor X1 approves.
If a n:licensor commissions the licensee X2 then there is a licensee X4 and it is necessary that the licensee X4 comments.
If it is false that the n:licensor X1 approves then it is false that it is admissible that the licensee X2 comments.
If it is false that it is admissible that the licensee X2 publishes

## Logic-based / Prolog

In [19]:
person(adam).
person(eve).
person(antony).
person(ruth).
person(lisa).

 % add Licensee?
publish(time, 24).

% Variables start with Uppercase!!!
% Art 1 - Add sth about publishing to Art 1?

made_offer(adam,eve).
accepts_offer(eve, adam).
paid_fee(eve, adam).
approves_comments(adam, eve).
commissions(adam,eve).

licensor(X):-
	licenses(X, Y).

licensee(Y):-
	licenses(X, Y).

licenses(X, Y) :-
	person(X), 
	person(Y),
	made_offer(X, Y),
	accepts_offer(Y, X),
	paid_fee(Y, X).

may_publish(Y) :-
licenses(X, Y),
	approves_comments(X,Y).

% Art 2
must_publish(Y) :-
licenses(X, Y),
	approves_comments(X,Y),
commissions(X,Y).

%Art 3
forbidden_publish(Y):-
	not(approves_comments(X, Y)).

must_remove_comments(Y,T):-
	license(X, Y),
	requested_removal(X,Y, T).

punisheable(Y, T1):-
	must_remove_comments(Y, T),
	T1 is T+24.


In [20]:
?- must_publish(eve).

true.

## Lexon

LEX: Evaluation License System.
LEXON: 0.2.12
PREAMBLE: This is a licensing contract for a software evaluation.
 
TERMS:

"Licensor" is [ a person ].
"Arbiter" is [ a person ].

"License" is this contract.

"Licensing Fee" is [ an amount ].
"Breach Fee" is [ an amount ].

The Licensor appoints the Arbiter,
fixes the Licensing Fee,
and fixes the Breach Fee.

CONTRACTS per Licensee:

"Description of Goods" is [ a text ].
"Licensee" is [ a person ].
"Paid" is [ a binary ].
"Commissioned" is [ a binary ].
"Comment Text" is [ a text ].
"Published" is [ a binary ].
"Permission to Comment" is [ a binary ].

The Licensor fixes the Description of Goods.

CLAUSE: Pay.
The Licensee pays the Licensing Fee to the Licensor,
and the Breach Fee into escrow.
The License is therefore Paid.
 
CLAUSE: Commission.
The Licensor may certify this License as Commissioned.

CLAUSE: Comment.
The Licensee may register a Comment Text.

CLAUSE: Publication.
The Licensee may certify the License as Published.

CLAUSE: Grant Permission to Comment.
The Licensee may grant the Permission to Comment.

CLAUSE: Examine for Breach.
The Arbiter may Declare Breach, if:
the License is Commissioned and no Comment Text is set;
or the License is Published and there is no Permission to Comment.

CLAUSE: Declare Breach.
The Arbiter may pay the Breach Fee to the Licensor, 
and afterwards terminate this License.


In [34]:
./lexon

bash: ./lexon: No such file or directory



In [ ]:
lexon v2 vs lexon v3